In [53]:
USERNAME = "minhptn@sendo.vn"
PASSWORD = "Sendo@123"

In [54]:
import pickle
import glob
import os 
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime, timedelta

# Set download folder



def initialize_driver(web):
    if web == 'sos':
        download_folder = r'D:\sendo\Code\Downloaded\SOS'   
    elif web == 'tms':
        download_folder = r'D:\sendo\Code\Downloaded\TMS'   
    
    """Initialize the Selenium WebDriver with custom options."""
    global driver  # Declare the driver as global

    # Set up Chrome options
    chrome_driver = r"chromedriver-win64\chromedriver.exe"
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")  # Start the browser maximized
    chrome_options.add_argument('--disable-gpu')  # Disable GPU (useful for certain systems)
    chrome_options.add_argument('--no-sandbox')  # No sandbox mode (useful for some environments)
    chrome_options.add_argument('--disable-dev-shm-usage')  # Prevent shared memory usage issues

    # Configure download preferences
    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": download_folder,  # Automatically save files to this folder
        "download.prompt_for_download": False,          # Disable the file destination popup
        "download.directory_upgrade": True,             # Automatically overwrite existing files
        "plugins.always_open_pdf_externally": True,     # PDFs should be downloaded, not opened in the browser
        "safebrowsing.enabled": True,                   # Enable safe browsing
        "profile.default_content_setting_values.automatic_downloads": 1  # Allow multiple downloads
    })

    # Use Service to specify the ChromeDriver path
    service = Service(chrome_driver)

    # Initialize the WebDriver
    driver = webdriver.Chrome(service=service, options=chrome_options)
    print("Driver initialized.")

In [55]:
def login(site, username, password):
    global driver
    if site == 'sos':
        driver.get("https://sos.sendo.vn")
        home = "https://sos.sendo.vn"
    elif site == 'tms':
        driver.get("https://tms.sendo.vn")
        home = "https://tms.sendo.vn"

    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Đăng Nhập - Login')]"))
    )
    login_button.click()

    email_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//input[@type='email']"))
    )
    email_field.send_keys(USERNAME)
    email_field.send_keys(Keys.RETURN)

    password_field = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, "//input[@type='password']"))
    )
    password_field.send_keys(PASSWORD)
    password_field.send_keys(Keys.RETURN)

    WebDriverWait(driver, 120).until(
        EC.url_contains(home)
    )

    if site == 'sos':
        with open("cookies_sos.pkl", "wb") as file:
            pickle.dump(driver.get_cookies(), file)
        print("Logged in and cookies saved.")
    elif site == 'tms':
        with open("cookies_tms.pkl", "wb") as file:
            pickle.dump(driver.get_cookies(), file)
        print("Logged in and cookies saved.")


def login_with_cookies(site):
    global driver
    try:
        if site == 'sos':
            driver.get("https://sos.sendo.vn")
        
            with open("cookies_sos.pkl", "rb") as file:
                cookies = pickle.load(file)
                for cookie in cookies:
                    driver.add_cookie(cookie)
                    
            # Refresh to apply cookies
            driver.refresh()

            # Confirm element existence for SOS site
            existence_sos = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, "//div[@class='navbar-container d-flex content']"))
            )
            if existence_sos:
                print("Logged in to SOS using cookies.")
                return True

        elif site == 'tms':
            driver.get("https://tms.sendo.vn")

            with open("cookies_tms.pkl", "rb") as file:
                cookies = pickle.load(file)
                for cookie in cookies:
                    driver.add_cookie(cookie)
                    
            # Refresh to apply cookies
            driver.refresh()

            # Confirm element existence for TMS site
            existence_tms = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "page-header"))
            )
            if existence_tms:
                print("Logged in to TMS using cookies.")
                return True

        print("Site not recognized or failed to log in.")
        return False

    except Exception as e:
        print(f"Failed to log in using cookies: {e}")
        return False


In [56]:
# # # Initialize driver
# driver = webdriver.Chrome(service=service, options=chrome_options)

In [57]:
last_days_of_months = [
        'January 31', 'February 28', 'March 31', 'April 30', 'May 31', 'June 30', 
        'July 31', 'August 31', 'September 30', 'October 31', 'November 30', 'December 31']

def choose_hub(hub_name):
    select_hub = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "select2-selection__rendered"))
    )

    # Click the dropdown to open the options
    select_hub.click()
    time.sleep(1)

    # Wait until the specific hub option is displayed and click it
    hub_option = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.XPATH, f"//li[text()='{hub_name}']"))
    )
    hub_option.click()
    print(f"Selected the hub: {hub_name}.")
    time.sleep(1)


def choose_date_sorting(date1, date2):
    date_picker = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "fp-range"))
    )
    date_picker.click()
    time.sleep(1)

    day_month1 = date1.split(',')[0]

    # Check if date1 is the last date of the month
    if day_month1 in last_days_of_months:
        print('choose prev calendar month')
        prev_month_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'flatpickr-prev-month'))
        )
        prev_month_button.click()
        time.sleep(1)

    print('choose date 1')    
    specific_date = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, f"//span[contains(@class, 'flatpickr-day') and @aria-label='{date1}']"))
    )
    specific_date.click()


    specific_date = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, f"//span[contains(@class, 'flatpickr-day') and @aria-label='{date2}']"))
    )
    specific_date.click()

def choose_date_ob(date1, date2):
    try:
        # Wait for the date picker to be present and click it
        date_picker = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.ID, "fp-range-2"))
        )
        date_picker.click()

        day_month1 = date1.split(',')[0]
        # Check if date1 is the last date of the month
        if day_month1 in last_days_of_months:
            prev_month_button = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((
                        By.XPATH, 
            "//div[contains(@class, 'flatpickr-calendar') and contains(@class, 'rangeMode') and contains(@style, 'left: auto')]"
            "//span[contains(@class, 'flatpickr-prev-month')]")))
            prev_month_button.click()
            time.sleep(1)

        # Select the first date inside the div with 'style' containing 'left: auto'
        specific_date1 = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((
                By.XPATH, 
                f"//div[contains(@style, 'left: auto')]//span[contains(@class, 'flatpickr-day') and @aria-label='{date1}']"
            ))
        )
        specific_date1.click()
        

        # Select the second date inside the div with 'style' containing 'left: auto'
        specific_date2 = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((
                By.XPATH, 
                f"//div[contains(@style, 'left: auto')]//span[contains(@class, 'flatpickr-day') and @aria-label='{date2}']"
            ))
        )
        specific_date2.click()
    
    except Exception as e:
        print(f"Error selecting dates: {e}")
        return

def choose_date_manifest(date1, date2):
    date_picker = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "fp-range-2"))
    )
    date_picker.click()
    time.sleep(2)

    day_month1 = date1.split(',')[0]

    # Check if date1 is the last date of the month
    if day_month1 in last_days_of_months:
        print('choose prev calendar month')
        prev_month_button = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'flatpickr-prev-month'))
        )
        prev_month_button.click()
        time.sleep(1)

    print('choose date 1')    
    specific_date = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, f"//span[contains(@class, 'flatpickr-day') and @aria-label='{date1}']"))
    )
    specific_date.click()


    specific_date = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, f"//span[contains(@class, 'flatpickr-day') and @aria-label='{date2}']"))
    )
    specific_date.click()




def click_download_sorting():
    # Look for the search button by ID and click it
    search_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "search-button"))
    )
    search_button.click()
    time.sleep(1)

    danger_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'btn btn-outline-danger waves-effect')]"))
    )
    danger_button.click()
    time.sleep(1)

    # Wait for the small popup to appear and select the checkbox by ID
    popup_label = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "checkdetail2"))
    )
    popup_label.click()
    time.sleep(1)

    # Look for the primary confirmation button and click it
    confirm_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'btn btn-primary waves-effect waves-float waves-light')]"))
    )
    confirm_button.click()
    print("File downloading")
    time.sleep(5)

    # Switch back to the original tab after confirming the download
    original_tab = driver.window_handles[0]
    driver.switch_to.window(original_tab)


def click_download_ob():
    danger_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'btn btn-outline-danger ms-1 waves-effect')]"))
    )
    danger_button.click()
    time.sleep(1)

    confirm_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'btn btn-primary waves-effect waves-float waves-light')]"))
    )
    confirm_button.click()
    print("File downloading")
    time.sleep(5)

    # Switch back to the original tab after confirming the download
    original_tab = driver.window_handles[0]
    driver.switch_to.window(original_tab)


def click_download_manifest():
    danger_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'btn btn-outline-danger waves-effect')]"))
    )
    danger_button.click()
    time.sleep(1)

    confirm_button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'btn btn-primary waves-effect waves-float waves-light')]"))
    )

    # Use JavaScript to execute the button's onclick function directly
    driver.execute_script("view.exportData()")
    print("File downloading")

    # Switch back to the original tab after confirming the download
    original_tab = driver.window_handles[0]
    driver.switch_to.window(original_tab)


def download_export():
    logs=[]

    driver.get("https://sos.sendo.vn/exportlogs/index")

    # Wait for the table with ID 'export-datatable' to load
    table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "export-datatable"))
    )

    rows = table.find_elements(By.XPATH, ".//tr")

    # Get today's date in the required format
    today = datetime.now().strftime("%d/%m/%Y")

    for i, row in enumerate(reversed(rows[:6])):  # Limit to top 6 rows
        try:
            # Check if the row contains today's date
            if today not in row.text:
                continue

            # Find the download link
            link = row.find_element(By.XPATH, ".//a[contains(@href, 'http')]")
            download_url = link.get_attribute("href")

            file_name = download_url.split('/')[-1]
            logs.append(file_name)

            # Click the link to start the download (it will open a new tab)
            driver.execute_script("arguments[0].click();", link)
            print(f"Started downloading file from row {6-i}: {download_url}")

            time.sleep(1)

        except Exception as e:
            print(f"Could not process row {6-i}: {e}")
    
    return logs

            
def clear_download_folder(folder_path):
    files = glob.glob(os.path.join(folder_path, "*"))
    for file in files:
        os.remove(file)


def close_other_tabs():
    # Get all window handles (tabs)
    window_handles = driver.window_handles
    
    # Ensure there is more than one tab
    if len(window_handles) > 1:
        # Loop through all tabs except the first one
        for handle in window_handles[1:]:
            # Switch to the tab and close it
            driver.switch_to.window(handle)
            driver.close()
        
        # Switch back to the first tab
        driver.switch_to.window(window_handles[0])



    

In [58]:
import os
import glob
from datetime import datetime, timedelta
import zipfile

def collate_sorting(destination_dir):
    # Directories
    current_dir = 'Downloaded'

    # Get yesterday's date in the format 'ddmm'
    yesterday = datetime.now() - timedelta(days=1)
    yesterday_str = yesterday.strftime('%d%m')

    # Get today's date
    today = datetime.now().date()

    print(f"Yesterday's date: {yesterday_str}")

    # Expected filenames for sorting
    sorting_filenames = [
        f"{yesterday_str}_hsnb",
        f"{yesterday_str}_hbta",
        f"{yesterday_str}_hs12",
        f"{yesterday_str}_hba",
        f"{yesterday_str}_hbc",
        f"{yesterday_str}_hbb"
    ]

    # Get all .csv files in the 'Downloaded' directory
    csv_files = glob.glob(os.path.join(current_dir, '*.csv'))

    # Filter files created today
    csv_files_today = [
        file for file in csv_files if datetime.fromtimestamp(os.path.getctime(file)).date() == today
    ]

    print(f"CSV files created today: {len(csv_files_today)}")

    # Sort the files by creation date
    csv_files_today.sort(key=os.path.getctime)

    # Ensure that the number of sorting filenames matches the number of today's CSV files
    if len(csv_files_today) == len(sorting_filenames):
        for i in range(len(csv_files_today)):
            # Get the current file path
            current_file_path = csv_files_today[i]
            
            # Define the new file path with the new name
            new_file_name = os.path.join(destination_dir, sorting_filenames[i] + '.csv')
            
            # Rename the file
            os.rename(current_file_path, new_file_name)
            print(f'Renamed: {current_file_path} to {new_file_name}')
    else:
        print("The number of sorting filenames does not match the number of CSV files.")


def collate_ob(destination_dir):
    """
    Extracts .zip files created today in the 'Downloaded' directory,
    renames the extracted .csv file according to a predefined format,
    and deletes the .zip file after extraction.

    Parameters:
        destination_dir (str): The directory where the extracted and renamed files will be stored.
    """
    current_dir = 'Downloaded'

    yesterday = datetime.now() - timedelta(days=1)
    yesterday_str = yesterday.strftime('%d%m')

    today = datetime.now().date()

    print(f"Yesterday's date in 'ddmm' format: {yesterday_str}")

    # Expected filenames for renaming
    ob_filenames = [
        f"{yesterday_str}_hsnb.csv",
        f"{yesterday_str}_hbta.csv",
        f"{yesterday_str}_hs12.csv",
        f"{yesterday_str}_hba.csv",
        f"{yesterday_str}_hbc.csv",
        f"{yesterday_str}_hbb.csv"
    ]

    # Get all .zip files in the 'Downloaded' directory
    zip_files = glob.glob(os.path.join(current_dir, '*.zip'))

    # Filter .zip files created today
    zip_files_today = [
        file for file in zip_files if datetime.fromtimestamp(os.path.getctime(file)).date() == today
    ]

    # Sort the .zip files by creation time
    zip_files_today.sort(key=os.path.getctime)

    # Check if the number of .zip files matches the number of expected filenames
    if len(zip_files_today) == len(ob_filenames):
        for i, zip_file in enumerate(zip_files_today):
            with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                # Extract the single file from the .zip directly to destination_dir
                extracted_file = zip_ref.namelist()[0]
                zip_ref.extract(extracted_file, destination_dir)
                print(f"Extracted: {zip_file} to {destination_dir}")

            # Construct full paths for the extracted and renamed files
            extracted_file_path = os.path.join(destination_dir, extracted_file)
            renamed_file_path = os.path.join(destination_dir, ob_filenames[i])

            # Check if the renamed file already exists
            if os.path.exists(renamed_file_path):
                print(f"File already exists: {renamed_file_path}. Skipping.")
                # Remove the .zip file since it was processed
                os.remove(zip_file)
                print(f"Removed .zip file: {zip_file}")
                continue

            # Rename the extracted file
            os.rename(extracted_file_path, renamed_file_path)
            print(f"Renamed: {extracted_file_path} to {renamed_file_path}")

            # Remove the .zip file after extraction and renaming
            os.remove(zip_file)
            print(f"Removed .zip file: {zip_file}")
    else:
        print("The number of .zip files does not match the number of expected filenames.")


def collate_manifest(destination_dir):
    """
    Renames .csv files in the 'Downloaded' directory according to a predefined format
    and moves them to the specified destination directory.

    Parameters:
        destination_dir (str): The directory where sorted files will be moved.
    """
    # Directories
    current_dir = 'Downloaded'

    # Get yesterday's date in the format 'ddmm'
    yesterday = datetime.now() - timedelta(days=1)
    yesterday_str = yesterday.strftime('%d%m')

    # Get today's date
    today = datetime.now().date()

    print(f"Yesterday's date: {yesterday_str}")

    # Expected filenames for sorting
    manifest_filenames = [
        f"{yesterday_str}_hsnb",
        f"{yesterday_str}_hbta",
        f"{yesterday_str}_hs12",
        f"{yesterday_str}_hba",
        f"{yesterday_str}_hbc",
        f"{yesterday_str}_hbb"
    ]

    # Get all .csv files in the 'Downloaded' directory
    csv_files = glob.glob(os.path.join(current_dir, '*.csv'))

    # Filter files created today
    csv_files_today = [
        file for file in csv_files if datetime.fromtimestamp(os.path.getctime(file)).date() == today
    ]

    print(f"CSV files created today: {len(csv_files_today)}")

    # Sort the files by creation date
    csv_files_today.sort(key=os.path.getctime)

    # Ensure that the number of sorting filenames matches the number of today's CSV files
    if len(csv_files_today) == len(manifest_filenames):
        for i in range(len(csv_files_today)):
            # Get the current file path
            current_file_path = csv_files_today[i]
            
            # Define the new file path with the new name
            new_file_name = os.path.join(destination_dir, manifest_filenames[i] + '.csv')
            
            # Rename the file
            os.rename(current_file_path, new_file_name)
            print(f'Renamed: {current_file_path} to {new_file_name}')
    else:
        print("The number of manifest filenames does not match the number of CSV files.")


In [59]:
initialize_driver('tms')

if not login_with_cookies('tms'):
    print("Falling back to manual login.")
    login('tms', USERNAME, PASSWORD)

Driver initialized.
Failed to log in using cookies: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF76E556CB5+28821]
	(No symbol) [0x00007FF76E4C3840]
	(No symbol) [0x00007FF76E36578A]
	(No symbol) [0x00007FF76E3B91BE]
	(No symbol) [0x00007FF76E3B94AC]
	(No symbol) [0x00007FF76E402647]
	(No symbol) [0x00007FF76E3DF33F]
	(No symbol) [0x00007FF76E3FF412]
	(No symbol) [0x00007FF76E3DF0A3]
	(No symbol) [0x00007FF76E3AA778]
	(No symbol) [0x00007FF76E3AB8E1]
	GetHandleVerifier [0x00007FF76E88FCAD+3408013]
	GetHandleVerifier [0x00007FF76E8A741F+3504127]
	GetHandleVerifier [0x00007FF76E89B5FD+3455453]
	GetHandleVerifier [0x00007FF76E61BDBB+835995]
	(No symbol) [0x00007FF76E4CEB5F]
	(No symbol) [0x00007FF76E4CA814]
	(No symbol) [0x00007FF76E4CA9AD]
	(No symbol) [0x00007FF76E4BA199]
	BaseThreadInitThunk [0x00007FFD62D0259D+29]
	RtlUserThreadStart [0x00007FFD637AAF38+40]

Falling back to manual login.
Logged in and cookies saved.


In [60]:
def choose_date_shipment(date1, date2):
    day1 = date1.split()[1].replace(",", "")
    day2 = date2.split()[1].replace(",", "")

    date_picker = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "ExpectedDeliveryDateStr"))
)
    date_picker.click()
    time.sleep(4)

    day_month1 = date1.split(',')[0]

    # # Check if date1 is the last date of the month
    # if day_month1 in last_days_of_months:
    #     print('choose prev calendar month')
    #     prev_month_button = WebDriverWait(driver, 5).until(
    #         EC.presence_of_element_located((By.CLASS_NAME, 'flatpickr-prev-month'))
    #     )
    #     prev_month_button.click()
    #     time.sleep(1)

    print('choose date 1:', day1)    

    target_date1 = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable(
            (By.XPATH, f"//table[contains(@class, 'table-condensed')]//td[@class='available' and text()='{day1}']")
        )
    )
    target_date1.click()
    
    target_date2 = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable(
        (By.XPATH, f"//table[contains(@class, 'table-condensed')]//td[contains(@class, 'active start-date available') and text()='{day2}']")
    )
)
    target_date2.click()



def choose_shipment_type():
    og_type_element = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "og-type"))
    )
    og_type_element.click()

   # Wait for the checkbox with value="1" to become clickable and click it
    checkbox_1 = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.XPATH, "//ul[contains(@class, 'multiselect-container dropdown-menu')]//label[contains(., 'Đơn hàng không tách')]/input[@type='checkbox']")
        )
    )
    checkbox_1.click()

    checkbox_2 = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located(
            (By.XPATH, "//ul[contains(@class, 'multiselect-container dropdown-menu')]//label[contains(., 'Đơn hàng tách(con)')]/input[@type='checkbox']")
        )
    )
    checkbox_2.click()

    body_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )

    body_element.click()


def click_download_shipment():
    export_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable(
            (By.XPATH, "//button[contains(@onclick, 'sendo.OGshipment.showConfirmExportPopup()') and contains(., 'Xuất Vận Đơn')]")
        )
    )
    export_button.click()
    time.sleep(1)

    accept_button1 = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//input[@type='checkbox' and @id='chk_accept' and contains(@onchange, 'sendo.OGshipment.accept')]"))
    )
    accept_button1.click()

    accept_button2 = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@type='button' and @id='btn_accept' and contains(@onclick, 'sendo.OGshipment.showExportPopup')]"))
    )
    accept_button2.click()

    export_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@type='button' and contains(@onclick, 'sendo.OGshipment.export')]"))
    )
    export_button.click()

    # Switch back to the original tab after confirming the download
    time.sleep(7)
    original_tab = driver.window_handles[0]
    driver.switch_to.window(original_tab)




In [68]:
def choose_date_trip(date1, date2):
    # Wait for the date input field to become available
    date_picker = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "DateStr"))
    )
    
    # Clear the existing value in the date field
    date_picker.click()  # Activate the field
    date_picker.clear()
    
    # Input the new date range
    new_date_range = f"{date1} - {date2}"
    date_picker.send_keys(new_date_range)
    
    # Confirm the date selection (if needed, simulate pressing Enter or clicking outside)
    date_picker.send_keys(Keys.RETURN)
    time.sleep(2) 

    # day_month2 = date2.split(',')[0]

    # # Check if date1 is the last date of the month
    # if day_month1 in last_days_of_months:
    #     print('choose prev calendar month')
    #     prev_month_button = WebDriverWait(driver, 5).until(
    #         EC.presence_of_element_located((By.CLASS_NAME, 'flatpickr-prev-month'))
    #     )
    #     prev_month_button.click()
    #     time.sleep(1)


def click_download_trip():
    # Step 1: Click the "Xuất dữ liệu" button
    export_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@onclick='sendo.managetripshuttle.showConfirmExportPopup()' and contains(@class, 'btn btn yellow')]"))
    )
    export_button.click()
    print("Clicked 'Xuất dữ liệu' button.")

    # Step 2: Click the checkbox with id 'chk_accept'
    accept_checkbox = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "chk_accept"))
    )
    accept_checkbox.click()
    print("Checked the 'Tôi đã đọc và đồng ý' checkbox.")

    # Step 3: Click the "Đồng ý và tiếp tục" button
    accept_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "btn_accept"))
    )
    accept_button.click()
    print("Clicked 'Đồng ý và tiếp tục' button.")

    export_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//button[@type='button' and contains(@onclick, 'sendo.managetripshuttle.export')]"))
    )
    export_button.click()

    # Switch back to the original tab after confirming the download
    time.sleep(7)
    original_tab = driver.window_handles[0]
    driver.switch_to.window(original_tab)




In [69]:
driver.get('https://tms.sendo.vn/ManageTripShuttle')

now = datetime.now()
date1 = now.replace(day=1, hour=0, minute=0, second=0).strftime("%d/%m/%Y %H:%M")
print(date1)

yesterday = now - timedelta(days=1)
date2 = yesterday.replace(hour=23, minute=59, second=0).strftime("%d/%m/%Y %H:%M")
print(date2)

choose_date_trip(date1, date2)
click_download_trip()

01/12/2024 00:00
14/12/2024 23:59
Clicked 'Xuất dữ liệu' button.
Checked the 'Tôi đã đọc và đồng ý' checkbox.
Clicked 'Đồng ý và tiếp tục' button.


In [52]:
driver.get('https://tms.sendo.vn/OGShipment')

yesterday = datetime.now() - timedelta(days=1)
day = yesterday.day
my_date = yesterday.strftime(f"%B {day}, %Y")
print(my_date)

choose_date_shipment(my_date, my_date)
choose_shipment_type()
click_download_shipment()

December 14, 2024
choose date 1: 14


In [14]:
from datetime import datetime, timedelta
hub_names= ['HSNB - HUB Nhà Bè', 'HBTA - HUB Thuận An', 'HS12 - HUB Quận 12', 
            'HS09 - HUB Thủ Đức', 'HS08 - HUB Quận 8',  'HSBT - HUB Bình Tân']

# Calculate yesterday's date and format it
yesterday = datetime.now() - timedelta(days=1)
day = yesterday.day
my_date = yesterday.strftime(f"%B {day}, %Y")
print(my_date)

clear_download_folder(download_folder)
try:
    for hub_name in hub_names:
        driver.get("https://sos.sendo.vn/Commodities")
        print('go to Commodities')
        time.sleep(2)
        choose_hub(hub_name)
        time.sleep(1)
        choose_date_sorting(date1=my_date, date2=my_date)
        click_download_sorting()
        time.sleep(1)

    time.sleep(7)
    close_other_tabs()
    download_export()
    collate_sorting(r'D:\sendo\analysis\warehouse_tracking_t12_hub\Sorting')
    

    for hub_name in hub_names:
        driver.get("https://sos.sendo.vn/outbound/index")
        print('go to Outbound')
        time.sleep(2)
        choose_hub(hub_name)
        time.sleep(1)
        choose_date_ob(date1=my_date, date2=my_date)
        click_download_ob()
        time.sleep(1)
        
    time.sleep(7)    
    close_other_tabs()
    download_export()
    collate_ob(r'D:\sendo\analysis\warehouse_tracking_t12_hub\OB')
    
    for hub_name in hub_names:
        driver.get("https://sos.sendo.vn/ManifestNew")
        print('go to Manifest')
        time.sleep(2)
        choose_hub(hub_name)
        time.sleep(1)
        choose_date_manifest(date1=my_date, date2=my_date)
        time.sleep(1)
        click_download_manifest()
        time.sleep(1)
        
    time.sleep(16)
    close_other_tabs()
    download_export()
    

    
except Exception as e:
    print(f"An error occurred: {e}")

finally:
    print("Process completed.")



December 10, 2024
go to Commodities
Selected the hub: HSNB - HUB Nhà Bè.
choose date 1
File downloading
go to Commodities
Selected the hub: HBTA - HUB Thuận An.
choose date 1
File downloading
go to Commodities
Selected the hub: HS12 - HUB Quận 12.
choose date 1
File downloading
go to Commodities
Selected the hub: HS09 - HUB Thủ Đức.
choose date 1
File downloading
go to Commodities
Selected the hub: HS08 - HUB Quận 8.
choose date 1
File downloading
go to Commodities
Selected the hub: HSBT - HUB Bình Tân.
choose date 1
File downloading
Started downloading file from row 6: https://s3-sgn10.fptcloud.com/og-logistic-contract/202412/11_12_2024_xuat_chi_tiet_hang_hoa_20241210_20241210_1733876114009.csv?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Credential=LZD6NU68SS1UWD4JKLK7%2F20241211%2Fus-east-1%2Fs3%2Faws4_request&X-Amz-Date=20241211T001520Z&X-Amz-Expires=604800&X-Amz-SignedHeaders=host&response-content-disposition=attachment%3B%20filename%3D%22202412%2F11_12_2024_xuat_chi_tiet_hang_hoa_20241

In [7]:
from datetime import datetime, timedelta
hub_names= ['HSNB - HUB Nhà Bè', 'HBTA - HUB Thuận An', 'HS12 - HUB Quận 12', 
            'HS09 - HUB Thủ Đức', 'HS08 - HUB Quận 8',  'HSBT - HUB Bình Tân']

# Calculate yesterday's date and format it
yesterday = datetime.now() - timedelta(days=1)
day = yesterday.day
my_date = yesterday.strftime(f"%B {day}, %Y")
print(my_date)

for hub_name in hub_names:
    driver.get("https://sos.sendo.vn/ManifestNew")
    print('go to Manifest')
    time.sleep(2)
    choose_hub(hub_name)
    time.sleep(1)
    choose_date_manifest(date1=my_date, date2=my_date)
    time.sleep(1)
    click_download_manifest()
    time.sleep(1)
        
time.sleep(20)
close_other_tabs()
download_export()
collate_manifest(r'D:\sendo\analysis\Manifest_t12\t12')

December 11, 2024
go to Manifest


NameError: name 'choose_hub' is not defined

In [15]:
download_export()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.109)
Stacktrace:
	GetHandleVerifier [0x00007FF7892E6CB5+28821]
	(No symbol) [0x00007FF789253840]
	(No symbol) [0x00007FF7890F578A]
	(No symbol) [0x00007FF7890CF4F5]
	(No symbol) [0x00007FF789176247]
	(No symbol) [0x00007FF78918ECE2]
	(No symbol) [0x00007FF78916F0A3]
	(No symbol) [0x00007FF78913A778]
	(No symbol) [0x00007FF78913B8E1]
	GetHandleVerifier [0x00007FF78961FCAD+3408013]
	GetHandleVerifier [0x00007FF78963741F+3504127]
	GetHandleVerifier [0x00007FF78962B5FD+3455453]
	GetHandleVerifier [0x00007FF7893ABDBB+835995]
	(No symbol) [0x00007FF78925EB5F]
	(No symbol) [0x00007FF78925A814]
	(No symbol) [0x00007FF78925A9AD]
	(No symbol) [0x00007FF78924A199]
	BaseThreadInitThunk [0x00007FFD62D0259D+29]
	RtlUserThreadStart [0x00007FFD637AAF38+40]


In [8]:
collate_manifest(r'D:\sendo\analysis\Manifest_t12\t12')

NameError: name 'collate_manifest' is not defined

In [ ]:
import time
import win32com.client

# Path to the Excel file
excel_file = r"D:\sendo\analysis\warehouse_tracking_t12_hub\warehouse_tracking_hub_day_v3.xlsx"

# Open Excel using COM interface
excel = win32com.client.Dispatch("Excel.Application")
excel.Visible = True  # Set to True if you want to see the process

# Open the workbook
workbook = excel.Workbooks.Open(excel_file)

# Refresh all data connections in the workbook
print('refreshing')
workbook.RefreshAll()

while excel.CalculationState != 0:  # 0 means Excel is idle (no calculation or refresh in progress)
    print("Refreshing...")
    time.sleep(10) 

    
# Save the workbook
workbook.Save()
time.sleep(10)

# Close the workbook and Excel
workbook.Close(False)
# excel.Quit()

print("File refreshed and saved successfully.")


refreshing


com_error: (-2147418111, 'Call was rejected by callee.', None, None)